In [14]:
import numpy as np
import time
from tqdm import tqdm
import os
import sys
sys.path.append("/home/xie/NN")
import math 
import pathos
from itertools import product
from seal import *

['/home/xie/NN/CHE', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/xie/.local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/local/lib/python3.8/dist-packages/pyknp-0.4.5-py3.8.egg', '/usr/lib/python3/dist-packages', '/home/xie/.local/lib/python3.8/site-packages/IPython/extensions', '/home/xie/.ipython', '/home/xie/NN/CHE', '/home/xie/NN/CHE', '../CHE', '../CHE', '/home/xie/NN']


In [8]:
def weight_change(weight, parms):
    """
    :param weight: weight of conv layer
    :param parms: encryption parameter dictionary
    :return: changed weight
    """
    weight = weight.numpy()
    weight_temp = np.zeros(weight.shape).tolist()
    for outer, inner, i in product(range(weight.shape[0]), range(weight.shape[1]), range(weight.shape[2])):
        weight_temp[outer][inner][i] = parms['encoder'].encode(weight[outer][inner][i].item(), parms['scale'])
    return weight_temp

In [9]:
def bias_change(bias, parms):
    """
    :param bias: bias of conv layer
    :param parms: encryption parameter dictionary
    :return:
    """
    for i in range(bias.__len__()):
        bias[i] = parms['encoder'].encode(bias[i], parms['scale'])
    return bias

In [10]:
def coefficients(mu, sigma, gamma, beta, eps, name):
    """
    :param mu: BN mu
    :param sigma: BN sigma
    :param gamma: BN gamma
    :param beta: BN beta
    :param eps: BN eps
    :param name: BN paradigm, CBA or CAB
    :return: coefficients merging
    """
    if name == 'CBA':
        temp = gamma / np.sqrt(sigma + eps)
        a = np.power(temp, 2)
        b = 2 * (beta - temp * mu) * temp
        c = np.power((beta - temp * mu), 2)
    elif name == 'CAB':
        a = gamma / np.sqrt(sigma + eps)
        b = 0
        c = beta - a * mu
    return a, b / a, c / a

In [11]:
def weight_diag(weight):
    """
    :param weight: weight of FC layer
    :return: padded and diagonalization weight of FC layer
    """
    weight = np.array(weight)
    weight = np.pad(weight, pad_width=((0, 0), (0, weight.shape[0] - weight.shape[1])))
    W = []
    for i in range(weight.shape[0]):
        if i == 0:
            W.append(np.diag(weight, i).tolist())
        else:
            W.append(np.concatenate([np.diag(weight, -i), np.diag(weight, weight.shape[0] - i)]).tolist())
    return W

In [12]:
def count_rotation(m, f):
    """
    :param m: input size
    :param f: filter size
    :param p: padding number
    :return: rotation index
    """
    idx = []
    for i in range(f):
        start = i * m
        for j in range(f):
            if i == f - 1 & j == f - 1:
                continue
            a = start + j
            if j == f - 1:
                a = (i + 1) * m - 1
                idx.append(a + 1)
            else:
                idx.append(a + 1)
    assert len(idx) == pow(f, 2) - 1
    return idx

In [13]:
def multi_conv(conv_weight, conv_bias, inputs, f, mask, parms):
    """
    multiprocessing
    :param conv_weight: weight of each output channel
    :param conv_bias: bias of each output channel
    :param inputs: input feature maps (fixed)
    :param f: filter size (fixed)
    :param mask: mask (fixed)
    :param parms: encryption parameters (fixed)
    :return: result of each output channel
    """
    # compute element-wise and accumulation
    in_channel = conv_weight[0].__len__()
    Z = []
    for inner in range(in_channel):
        Y = []
        for i in range(f * f):
            parms['evaluator'].mod_switch_to_inplace(conv_weight[inner][i], inputs[inner][i].parms_id())
            Y.append(parms['evaluator'].multiply_plain(inputs[inner][i], conv_weight[inner][i]))
            Z.append(parms['evaluator'].add_many(Y))
    Z = parms['evaluator'].add_many(Z)
    parms['evaluator'].rescale_to_next_inplace(Z)

    # add bias
    parms['evaluator'].mod_switch_to_inplace(conv_bias, Z.parms_id())
    Z.scale(2 ** int(math.log2(conv_bias.scale())))
    parms['evaluator'].add_plain_inplace(Z, conv_bias)
    mask_encode = mask
    parms['evaluator'].mod_switch_to_inplace(mask_encode, Z.parms_id())
    parms['evaluator'].multiply_plain_inplace(Z, mask_encode)
    parms['evaluator'].rescale_to_next_inplace(Z)
    
    print("hello world")

    return Z


def convolution(input, conv_weight, conv_bias, s, parms):

    out_channels = []
    in_channel = conv_weight[0].__len__()
    out_channel = conv_weight.__len__()
    f = int(math.sqrt(conv_weight[0][0].__len__()))
    p = 0
    parms['n'] = int((parms['m'] + 2 * p - f) / s + 1)
    y = parms["context"].key_context_data().parms()
    
    # search rotation index
    idx = count_rotation(parms['m'], f)
    for i in range(idx.__len__()):
        idx[i] = parms["valid_vector"][idx[i]]

    # valid values
    valid_index = []
    mask = [0 for i in range(parms['slots'])]
    for i in range(parms['n']):
        for j in range(parms['n']):
            valid_index.append(i * s * parms['m'] + j * s)
            mask[parms["valid_vector"][i * s * parms['m'] + j * s]] = 1
    mask = parms['encoder'].encode(mask, parms['scale'])
    mask.set_parms(y)
    inputs = [[0 for i in range(f * f)] for j in range(in_channel)]

    for i in range(in_channel):
        inputs[i][0] = input[i]
        inputs[i][0].set_parms(y)
        for j in range(f * f):
            inputs[i][j] = parms['evaluator'].rotate_vector(input[i], idx[j - 1], parms['galois_keys'])
            inputs[i][j].set_parms(y)
    
    print("Parallel-conv start")
    pool = pathos.multiprocessing.ProcessPool()
    inputs_s = [inputs] * out_channel
    f_s = [f] * out_channel
    mask_s = [mask] * out_channel
    parms_s = [parms] * out_channel
    

    result = pool.amap(multi_conv, (conv_weight, conv_bias, inputs_s, f_s, mask_s, parms_s)).get()
    pool.close()
    pool.join()

    # out_channels = result

    # set output of layer as input of next layer
    parms['m'] = parms['n']
    # set invalid vector
    Z = []
    for i in valid_index:
        Z.append(parms["valid_vector"][i])
    parms["valid_vector"] = Z
    return result

In [14]:
def bn_act(x, b, c, parms):
    channels = len(x)
    X = []
    for i in range(channels):
        # HE square
        # x^2
        x_2 = parms['evaluator'].square(x[i])
        parms['evaluator'].relinearize_inplace(x_2, parms['relin_keys'])
        parms['evaluator'].rescale_to_next_inplace(x_2)

        # b * x
        b_prime = parms['encoder'].encode(b[i].item(), parms['scale'])
        parms['evaluator'].mod_switch_to_inplace(b_prime, x[i].parms_id())
        b_x = parms['evaluator'].multiply_plain(x[i], b_prime)
        parms['evaluator'].rescale_to_next_inplace(b_x)

        c_prime = parms['encoder'].encode(c[i].item(), parms['scale'])

        parms['evaluator'].mod_switch_to_inplace(c_prime, x_2.parms_id())
        parms['evaluator'].mod_switch_to_inplace(b_x, x_2.parms_id())

        x_2.scale(2 ** int(math.log2(c_prime.scale())))
        b_x.scale(2 ** int(math.log2(c_prime.scale())))

        X.append(parms['evaluator'].add_plain(parms['evaluator'].add(x_2, b_x), c_prime))
    return X

In [ ]:
def multi_pack(x, pamrs):
    # HE rotation
    # idx: valuable idx; rot_idx: rotation index
    idx = parms["valid_vector"]
    # generate all-zeroes mask
    mask = [0 for h in range(parms['slots'])]
    mask[0] = 1
    mask = parms['encoder'].encode(mask, parms['scale'])
    parms['evaluator'].mod_switch_to_inplace(mask, x.parms_id())
    Z = []
    Z.append(parms['evaluator'].multiply_plain(x, mask))
    for k in range(self.parms['n'] * parms['n'] - 1):
        # HE element-wise multiplication and addition
        mask = [0 for h in range(parms['slots'])]
        mask[idx[k + 1]] = 1
        mask = parms['encoder'].encode(mask, parms['scale'])
        parms['evaluator'].mod_switch_to_inplace(mask, x.parms_id())
        Z.append(
            parms['evaluator'].rotate_vector(
                parms['evaluator'].multiply_plain(x, mask),
            rot_idx[k], parms['galois_keys']
            )
        )
    x = parms['evaluator'].add_many(Z)
        
    return parms['evaluator'].rotate_vector(x, -(n * n * i), parms['galois_keys'])


def pack_vectors(x, n, parms):
    channels = x.__len__()
    Z = []
    # Y.append(x[0])
    rot_idx = []
    for j in range(1, n * n):
        rot_idx.append(parms["valid_vector"][j] - j)
    y = parms["context"].key_context_data().parms()
    for i in range(channels):
        x[i].set_parms(y)
    pool = pathos.multiprocessing.ProcessPool()
    parms_s = [parms] * channels
    
    result = pool.amap(multi_pack, (x, parms_s)).get()
    pool.close()
    pool.join()
    
    Z = parms['evaluator'].add_many(result)
    parms['evaluator'].rescale_to_next_inplace(Z)
    return Z

In [ ]:
def fully_connected(self, x, weight, bias):
    Y = []
    temp = x

    rot_single = weight.__len__() - bias.__len__() + 1

    for i in range(weight.__len__()):

        w = parms['encoder'].encode(weight[i], parms['scale'])
        parms['evaluator'].mod_switch_to_inplace(w, x.parms_id())
        Y.append(
            parms['evaluator'].multiply_plain(temp, w)
        )
        if i != 0 and i < rot_single:
            temp = parms['evaluator'].rotate_vector(x, i + 1, parms['galois_keys'])

        B = []
        if i != weight.__len__() - 1 and i >= rot_single:
            B.append(parms['evaluator'].rotate_vector(x, i + 1, self.parms['galois_keys']))
            B.append(parms['evaluator'].rotate_vector(x, -weight.__len__() + i + 1, parms['galois_keys']))
            temp = parms['evaluator'].add_many(B)
    Y = parms['evaluator'].add_many(Y)
    parms['evaluator'].rescale_to_next_inplace(Y)

    B = parms['encoder'].encode(bias, parms['scale'])

    parms['evaluator'].mod_switch_to_inplace(B, Y.parms_id())
    Y.scale(2 ** int(math.log2(B.scale())))
    return parms['evaluator'].add_plain(Y, B)